# Unsupervised Birdsong Syllable Pipeline — Notebook Skeleton
_Created: 2025-11-07T00:58:12_
## Overview
This notebook scaffolds an end‑to‑end, label‑free pipeline:
1. Load audio → spectrogram
2. Event detection (connected components)
3. Split–merge clustering to form templates
4. Template merging (normalized L2 threshold)
5. Greedy matching pursuit over time × frequency with non‑overlap collar
6. Evaluation and simple visualizations

Fill in TODOs with species‑specific details and thresholds.


## 0. Setup

In [2]:
# Imports
import os
from dataclasses import dataclass
from pathlib import Path
from typing import List, Tuple, Dict, Any

import numpy as np
import scipy.signal as sps
import scipy.ndimage as ndi

# If you have librosa installed, uncomment
import librosa

# Plotting
import matplotlib.pyplot as plt

# Clustering (install hdbscan if available)
try:
    import hdbscan
except Exception as e:
    hdbscan = None
    print("HDBSCAN not available. Install with `pip install hdbscan`.")

# Random seed for reproducibility
np.random.seed(0)

# Paths
DATA_DIR = Path("../data/raw/")                # TODO: set to your audio folder
WORK_DIR = Path("../data/interim/")               # artifacts go here
WORK_DIR.mkdir(parents=True, exist_ok=True)

# Audio parameters
SR = 30000                               # sample rate Hz (TODO: set for your data)
N_FFT = 1024
HOP = 256
FMIN = 300
FMAX = 10000

# Detection parameters
THRESH_DB = 10.0                         # energy threshold in dB above noise floor
MIN_DUR_S = 0.015                        # min syllable duration
MAX_DUR_S = 0.5                          # max syllable duration
MIN_GAP_S = 0.005                        # collar/gap for non-overlap

# Clustering parameters
MIN_CLUSTER_SIZE = 10
MAX_CLUSTER_SIZE = 200
MERGE_L2_CUTOFF = 0.33                   # normalized L2 threshold for template merge

# Matching pursuit parameters
MAX_ITERS = 1                            # number of global passes
COLLAR_FRAMES = 2                        # non-overlap collar in frames

print("Configured.")

Configured.


## 1. Utilities

In [3]:
def read_wav_mono(path: Path, sr: int) -> np.ndarray:
    """Load audio file as mono float32 at target sr. TODO: replace with librosa if available."""
    # Placeholder: generate silence for skeleton use
    # Implement: y, _ = librosa.load(path.as_posix(), sr=sr, mono=True)
    y = np.zeros(sr * 2, dtype=np.float32)  # 2 seconds of silence
    return y

def power_to_db(S: np.ndarray, ref: float = 1.0, amin: float = 1e-10) -> np.ndarray:
    S = np.maximum(S, amin)
    return 10.0 * np.log10(S / ref)

def stft_spectrogram(y: np.ndarray, sr: int, n_fft: int, hop: int) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Return magnitude spectrogram (power), freqs, times."""
    f, t, Zxx = sps.stft(y, fs=sr, nperseg=n_fft, noverlap=n_fft - hop, boundary=None, padded=False)
    S = np.abs(Zxx) ** 2
    return S, f, t

def bandpass_mask(f: np.ndarray, fmin: float, fmax: float) -> np.ndarray:
    return (f >= fmin) & (f <= fmax)

def normalize_template(T: np.ndarray) -> np.ndarray:
    norm = np.linalg.norm(T.ravel()) + 1e-12
    return T / norm

def normalized_l2(a: np.ndarray, b: np.ndarray) -> float:
    a_n = normalize_template(a)
    b_n = normalize_template(b)
    return np.linalg.norm(a_n - b_n)

## 2. Event detection

In [4]:
from dataclasses import dataclass

@dataclass
class Event:
    t0: int
    t1: int
    f0: int
    f1: int

def detect_events(S_db: np.ndarray, thresh_db: float, min_dur_frames: int) -> List[Event]:
    """Connected components in time-frequency above threshold."""
    mask = S_db > thresh_db
    labeled, nlab = ndi.label(mask)
    events: List[Event] = []
    for lab in range(1, nlab + 1):
        ys, xs = np.where(labeled == lab)
        if xs.size == 0 or ys.size == 0:
            continue
        t0, t1 = xs.min(), xs.max() + 1
        f0, f1 = ys.min(), ys.max() + 1
        if (t1 - t0) >= min_dur_frames:
            events.append(Event(t0, t1, f0, f1))
    return events

# TODO: refine threshold using noise floor estimate


## 3. Feature extraction for clustering

In [5]:
def extract_event_patch(S: np.ndarray, ev: Event, pad_t: int = 4, pad_f: int = 4) -> np.ndarray:
    t0 = max(0, ev.t0 - pad_t); t1 = min(S.shape[1], ev.t1 + pad_t)
    f0 = max(0, ev.f0 - pad_f); f1 = min(S.shape[0], ev.f1 + pad_f)
    patch = S[f0:f1, t0:t1]
    return patch

def resize_to_box(patch: np.ndarray, tgt_shape: Tuple[int, int] = (64, 64)) -> np.ndarray:
    """Resize using simple zoom to fixed box. TODO: replace with better interpolation."""
    zoom_f = (tgt_shape[0] / patch.shape[0], tgt_shape[1] / patch.shape[1])
    return ndi.zoom(patch, zoom_f, order=1)

def patches_to_features(patches: List[np.ndarray]) -> np.ndarray:
    """Global PCA placeholder: flatten vectors. TODO: add PCA if needed."""
    flat = [p.astype(np.float32).ravel() for p in patches]
    X = np.vstack(flat) if flat else np.zeros((0, 4096), dtype=np.float32)
    return X

## 4. Split–merge clustering

In [6]:
def hdbscan_cluster(X: np.ndarray, min_size: int, max_size: int) -> np.ndarray:
    if hdbscan is None or X.shape[0] == 0:
        return np.array([], dtype=int)
    clusterer = hdbscan.HDBSCAN(min_cluster_size=min_size, min_samples=None)
    labels = clusterer.fit_predict(X)
    return labels

def make_templates(patches: List[np.ndarray], labels: np.ndarray) -> Dict[int, np.ndarray]:
    templates: Dict[int, np.ndarray] = {}
    for lab in np.unique(labels):
        if lab < 0:
            continue
        group = [p for p, l in zip(patches, labels) if l == lab]
        if not group:
            continue
        # Median image as template
        stack = np.stack([resize_to_box(p) for p in group], axis=0)
        T = np.median(stack, axis=0)
        templates[int(lab)] = normalize_template(T)
    return templates

def merge_templates(templates: Dict[int, np.ndarray], cutoff: float) -> Dict[int, np.ndarray]:
    """Agglomerative merge by normalized L2 distance with simple greedy pass."""
    labs = list(templates.keys())
    keep = {lab: True for lab in labs}
    rep = {lab: lab for lab in labs}
    for i, li in enumerate(labs):
        if not keep[li]: 
            continue
        for lj in labs[i+1:]:
            if not keep.get(lj, False):
                continue
            d = normalized_l2(templates[li], templates[lj])
            if d < cutoff:
                # merge lj into li
                keep[lj] = False
                rep[lj] = li
    merged = {}
    for lj, ok in keep.items():
        if ok:
            merged[lj] = templates[lj]
    return merged

## 5. Greedy matching pursuit

In [7]:
@dataclass
class Placement:
    t: int
    f: int
    lab: int
    score: float

def match_score(S: np.ndarray, T: np.ndarray, t: int, f: int) -> float:
    H, W = T.shape
    if f + H > S.shape[0] or t + W > S.shape[1]:
        return -np.inf
    window = S[f:f+H, t:t+W]
    # cosine similarity proxy via normalized L2
    denom = (np.linalg.norm(window.ravel()) * np.linalg.norm(T.ravel())) + 1e-10
    return float(np.dot(window.ravel(), T.ravel()) / denom)

def greedy_decompose(S: np.ndarray, templates: Dict[int, np.ndarray], collar_frames: int = 2, max_iters: int = 1) -> List[Placement]:
    placed: List[Placement] = []
    busy = np.zeros(S.shape[1], dtype=bool)  # time busy mask
    for _ in range(max_iters):
        improved = False
        for lab, T in templates.items():
            H, W = T.shape
            for t in range(0, S.shape[1] - W):
                if busy[max(0, t-collar_frames):min(S.shape[1], t+W+collar_frames)].any():
                    continue
                # naive frequency search
                for f in range(0, S.shape[0] - H):
                    sc = match_score(S, T, t, f)
                    if sc > 0.6:  # TODO: tune threshold
                        placed.append(Placement(t=t, f=f, lab=lab, score=sc))
                        busy[max(0, t):min(S.shape[1], t+W)] = True
                        improved = True
                        break
        if not improved:
            break
    return placed

## 6. Evaluation scaffolding

In [8]:
def bag_of_syllables(placements: List[Placement]) -> Dict[int, int]:
    counts: Dict[int, int] = {}
    for p in placements:
        counts[p.lab] = counts.get(p.lab, 0) + 1
    return counts

def simple_precision(tp: int, fp: int) -> float:
    return tp / max(1, (tp + fp))

# TODO: add matching to any available ground truth if present.


## 7. Visualization helpers

In [9]:
def show_spectrogram(S_db: np.ndarray, f: np.ndarray, t: np.ndarray, title: str = "Spectrogram (dB)") -> None:
    plt.figure()
    plt.imshow(S_db, aspect='auto', origin='lower', extent=[t.min(), t.max(), f.min(), f.max()])
    plt.xlabel("Time (s)")
    plt.ylabel("Frequency (Hz)")
    plt.title(title)
    plt.colorbar(label="dB")
    plt.show()

def overlay_placements(S_db: np.ndarray, f: np.ndarray, t: np.ndarray, placements: List[Placement], templates: Dict[int, np.ndarray]) -> None:
    plt.figure()
    plt.imshow(S_db, aspect='auto', origin='lower', extent=[t.min(), t.max(), f.min(), f.max()])
    for p in placements:
        T = templates[p.lab]
        H, W = T.shape
        t0 = t[p.t]; t1 = t[min(p.t + W, len(t)-1)]
        f0 = f[p.f]; f1 = f[min(p.f + H, len(f)-1)]
        plt.gca().add_patch(plt.Rectangle((t0, f0), (t1 - t0), (f1 - f0), fill=False, linewidth=1))
    plt.xlabel("Time (s)")
    plt.ylabel("Frequency (Hz)")
    plt.title("Placements overlay")
    plt.show()

## 8. End‑to‑end runner

In [10]:
def run_on_file(path: Path) -> Dict[str, Any]:
    # 1) Load
    y = read_wav_mono(path, SR)
    # 2) Spectrogram
    S, freqs, times = stft_spectrogram(y, SR, N_FFT, HOP)
    bp = bandpass_mask(freqs, FMIN, FMAX)
    S = S[bp, :]
    S_db = power_to_db(S, ref=np.max(S)+1e-12)
    # 3) Detect events
    min_dur_frames = int(MIN_DUR_S * SR / HOP)
    events = detect_events(S_db, THRESH_DB, min_dur_frames)
    # 4) Patches and features
    patches = [extract_event_patch(S_db, ev) for ev in events]
    patches = [resize_to_box(p) for p in patches if p.size > 0]
    X = patches_to_features(patches)
    # 5) Cluster and template
    labels = hdbscan_cluster(X, MIN_CLUSTER_SIZE, MAX_CLUSTER_SIZE)
    templates = make_templates(patches, labels) if labels.size else {}
    templates = merge_templates(templates, MERGE_L2_CUTOFF) if templates else {}
    # 6) Matching pursuit
    placements = greedy_decompose(S_db, templates, COLLAR_FRAMES, MAX_ITERS) if templates else []
    # 7) Outputs
    bos = bag_of_syllables(placements)
    return {
        "events": events,
        "patches": patches,
        "X_shape": X.shape,
        "labels": labels,
        "templates": templates,
        "placements": placements,
        "bag_of_syllables": bos,
        "S_db": S_db,
        "freqs": freqs[bp],
        "times": times,
    }

# Example usage (disabled by default)
result = run_on_file(WORK_DIR / "interesting.flac")  # TODO: set path
show_spectrogram(result["S_db"], result["freqs"], result["times"])

/home/jaeho/NYU/budgie/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/jaeho/NYU/budgie/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


ValueError: k must be less than or equal to the number of training points

## 9. TODOs
- Replace the dummy audio loader with librosa or soundfile.
- Add PCA before HDBSCAN and a second per‑cluster PCA+HDBSCAN split pass.
- Improve thresholding with local noise floor estimation.
- Tune template merge cutoff and matching threshold per species.
- Add evaluation against any available annotations.
- Persist artifacts: templates, placements, and embeddings.
